## <div align="center"> High Quality Growth Portfolio Generator

### Import Libraries / Load Data

In [1]:
import json
import numpy as np
import pandas as pd
import re
import requests

from bs4 import BeautifulSoup
from scipy import stats
from statistics import mean

In [2]:
# Import List of Candidate Stocks (S&P 500)
stocks = pd.read_csv('sp_500_stocks.csv')

### Define Data Scraping Functions / Helper Functions

In [3]:
# Create a function that determines the correct URL for a given ticker symbol and then generates the corresponding get request

def get_val_request(symbol):
    url_value_NYS = f'http://financials.morningstar.com/valuate/valuation-history.action?&t=XNYS:{symbol}&region=usa&culture=en-US&cur=&type=price-earnings&'
    url_value_NAS = f'http://financials.morningstar.com/valuate/valuation-history.action?&t=XNAS:{symbol}&region=usa&culture=en-US&cur=&type=price-earnings&'
    r_NYS = requests.get(url_value_NYS)
    r_NAS = requests.get(url_value_NAS)
    r = (r_NYS if r_NYS.text else r_NAS)
        
    return r

def get_deb_request(symbol):
    url_NYS = f'http://financials.morningstar.com/finan/financials/getKeyStatPart.html?&callback=jsonp1626044235897&t=XNYS:{symbol}&region=usa&culture=en-US&cur=&order=asc&_=1626044237707'
    url_NAS = f'http://financials.morningstar.com/finan/financials/getKeyStatPart.html?&callback=jsonp1626044235897&t=XNAS:{symbol}&region=usa&culture=en-US&cur=&order=asc&_=1626044237707'
    r_NYS = requests.get(url_NYS)
    r_NAS = requests.get(url_NAS)
    r = (r_NYS if r_NYS.text else r_NAS)
    
    return r

def get_growth_request(symbol):
    url_NYS = f'http://financials.morningstar.com/ajax/ReportProcess4HtmlAjax.html?&t=XNYS:{symbol}&region=usa&culture=en-US&cur=&reportType=is&period=12&dataType=A&order=asc&columnYear=5&curYearPart=1st5year&rounding=3&view=raw&r=783414&callback=jsonp1626764458821&_=1626764460992'
    url_NAS = f'http://financials.morningstar.com/ajax/ReportProcess4HtmlAjax.html?&t=XNAS:{symbol}&region=usa&culture=en-US&cur=&reportType=is&period=12&dataType=A&order=asc&columnYear=5&curYearPart=1st5year&rounding=3&view=raw&r=783414&callback=jsonp1626764458821&_=1626764460992'
    r_NYS = requests.get(url_NYS)
    r_NAS = requests.get(url_NAS)
    r = (r_NYS if r_NYS.text else r_NAS)
    
    return r

In [4]:
# Function that takes a ticker symbol, get_request, and the ratio type as arguments, and returns the current ratio value
def current_value_ratio(symbol, request, ratio_type):
    data_list = []
    ratio_text_dict = {'pe' : 'Price/Earnings', 'pb' : 'Price/Book', 'ps' : 'Price/Sales'}
    ratio_text = ratio_text_dict[ratio_type]
    soup = BeautifulSoup(request.text)   
    line = soup.find('th', {'abbr' : f'{ratio_text} for {symbol}'})
    
    try:
        while line.next_sibling.next_sibling != None:
            line = line.next_sibling.next_sibling
            try:
                data_list.append(float(line(text=True)[0]))
            except ValueError:
                data_list.append(np.NaN)
    except AttributeError:
        data_list.append(np.NaN)
        
    return data_list[-1]

# Function that takes ticker symbol and get request as arguments, and returns current debt / equity ratio
def current_debteq_ratio(symbol, request):
    data_list = []
    r = request.text.encode("utf-8").decode("unicode-escape")
    soup = BeautifulSoup(r)
    line = soup.find('th', {'id' : 'i68'})
    
    try:
        while line.next_sibling != None:
            line = line.next_sibling
            try:
                data_list.append(float(line(text=True)[0]))
            except ValueError:
                data_list.append(np.NaN)
    except AttributeError:
        data_list.append(np.NaN)
        
    return data_list[-1]

# Function that takes ticker symbol and get request as arguments, and returns a list of growth parameters
def growth_list(symbol, request):
    r = request.text.encode("utf-8").decode("unicode-escape")
    soup = BeautifulSoup(r)
    line = soup.find('th', {'id' : 'i68'})
    
    soup = BeautifulSoup(r)
    line = soup.find('div', {'id' : 'data_i30'})
    data = line.get_text('\t')
    growth_list_1 = data.split('\t')

    line = soup.find('div', {'id' : 'label_i86'})
    data = line.get_text('\t')
    growth_list_2 = data.split('\t')
    
    data_list = [growth_list_1, growth_list_2]
        
    return data_list

# Function that converts data from source format to int ( ex. input: '(1,234)' -> output: -1234 )
def data_to_int(data):
    for ind, symbol in enumerate(['(', ',', ')']):
        replacement = ['-', '', '']
        data = data.replace(symbol, replacement[ind])
    data = int(data)
    return data

### Build Dataframe

In [5]:
df_columns = ['Ticker', 'P/E Ratio', 'P/E Percentile', 'P/B Ratio', 'P/B Percentile', 'P/S Ratio', 
              'P/S Percentile', 'D/E Ratio', 'D/E Percentile', '% Change in Revenue (1-Year)',
              '1 Year Revenue Change Percentile', '% Change in Revenue (3-Year)', '3 Year Revenue Change Percentile',
              '% Change in EBITDA (1-Year)', '1 Year EBITDA Change Percentile', '% Change in EBITDA (3-Year)',
              '3 Year EBITDA Change Percentile', 'Value Score', 'Growth Score', 'Total Score']
df = pd.DataFrame(columns=df_columns)

for symbol in stocks['Ticker']:
    EF_1 = EF_2 = 0
    r_val = get_val_request(symbol)
    pe_ratio = current_value_ratio(symbol, r_val, 'pe')
    pb_ratio = current_value_ratio(symbol, r_val, 'pb')
    ps_ratio = current_value_ratio(symbol, r_val, 'ps')
    
    r_deb = get_deb_request(symbol)
    de_ratio = current_debteq_ratio(symbol, r_deb)
    
    r_growth = get_growth_request(symbol)
    
    try:
        data_list_2 = growth_list(symbol, r_growth)[1]
        
        RE_CY = (data_to_int(data_list_2[13]) if data_list_2[13] != '—' else np.NaN)
        RE_1Y = (data_to_int(data_list_2[12]) if data_list_2[12] != '—' else np.NaN)  
        RE_3Y = (data_to_int(data_list_2[10]) if data_list_2[10] != '—' else np.NaN) 
    except AttributeError:
        EF_1 = 1    # Set Error Flag
    try:
        data_list = growth_list(symbol, r_growth)[0]
        
        EB_CY = (data_to_int(data_list[160]) if data_list[160] != '—' else np.NaN) 
        EB_1Y = (data_to_int(data_list[159]) if data_list[159] != '—' else np.NaN) 
        EB_3Y = (data_to_int(data_list[157]) if data_list[157] != '—' else np.NaN) 
    except AttributeError:
        EF_2 = 1    # Set Error Flag
    
    RE_1Y_CH = ((((RE_CY - RE_1Y) / RE_1Y) * 100) if (RE_CY > 0 and RE_1Y > 0 and EF_1 == 0) else np.NaN) 
    RE_3Y_CH = ((((RE_CY - RE_3Y) / RE_3Y) * 100) if (RE_CY > 0 and RE_3Y > 0 and EF_1 == 0) else np.NaN) 
    
    EB_1Y_CH = ((((EB_CY - EB_1Y) / EB_1Y) * 100) if (EB_CY > 0 and EB_1Y > 0 and EF_2 == 0) else np.NaN) 
    EB_3Y_CH = ((((EB_CY - EB_3Y) / EB_3Y) * 100) if (EB_CY > 0 and EB_3Y > 0 and EF_2 == 0) else np.NaN)
    
    df = df.append(
            pd.Series(
                [
                    symbol,
                    pe_ratio,
                    'N/A',
                    pb_ratio,
                    'N/A',
                    ps_ratio,
                    'N/A',
                    de_ratio,
                    'N/A',
                    RE_1Y_CH,
                    'N/A',
                    RE_3Y_CH,
                    'N/A',
                    EB_1Y_CH,
                    'N/A',
                    EB_3Y_CH,
                    'N/A',
                    'N/A',
                    'N/A',
                    'N/A'
                ],
                index = df_columns
            ),
            ignore_index = True
        )
    
df

c:\users\amsg0\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: invalid escape sequence '\/'
c:\users\amsg0\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: invalid escape sequence '\/'


,Ticker,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,D/E Ratio,D/E Percentile,% Change in Revenue (1-Year),1 Year Revenue Change Percentile,% Change in Revenue (3-Year),3 Year Revenue Change Percentile,% Change in EBITDA (1-Year),1 Year EBITDA Change Percentile,% Change in EBITDA (3-Year),3 Year EBITDA Change Percentile,Value Score,Growth Score,Total Score
0,AAPL,28.7,N/A,37.8,N/A,7.2,N/A,1.65,N/A,5.512080,N/A,19.753178,N/A,-1.026142,N/A,5.813058,N/A,N/A,N/A,N/A
1,MSFT,36.0,N/A,15.3,N/A,13.1,N/A,0.42,N/A,17.531727,N/A,52.308808,N/A,24.423074,N/A,72.099135,N/A,N/A,N/A,N/A
2,AMZN,58.8,N/A,14.9,N/A,3.9,N/A,0.93,N/A,37.623431,N/A,117.053287,N/A,36.694768,N/A,216.612943,N/A,N/A,N/A,N/A
3,FB,5.8,N/A,1.1,N/A,5.7,N/A,7.08,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A,N/A,N/A
4,GOOGL,29.6,N/A,7.6,N/A,9.1,N/A,0.11,N/A,12.770532,N/A,64.653827,N/A,20.207354,N/A,80.945144,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,FOX,26.0,N/A,1.9,N/A,2.8,N/A,0.25,N/A,10.661698,N/A,43.167921,N/A,32.349949,N/A,130.469442,N/A,N/A,N/A,N/A
501,DISCA,14.0,N/A,1.3,N/A,1.7,N/A,1.25,N/A,-4.244436,N/A,55.259712,N/A,-6.693627,N/A,159.542281,N/A,N/A,N/A,N/A
502,UAA,31.4,N/A,6.1,N/A,2.1,N/A,0.85,N/A,-15.037023,N/A,-10.086397,N/A,NaN,N/A,NaN,N/A,N/A,N/A,N/A
503,UA,26.9,N/A,5.3,N/A,1.8,N/A,0.85,N/A,-15.037023,N/A,-10.086397,N/A,NaN,N/A,NaN,N/A,N/A,N/A,N/A


In [6]:
# Fill in missing values

for column in ['P/E Ratio', 'P/B Ratio', 'P/S Ratio', 'D/E Ratio', '% Change in Revenue (1-Year)', '% Change in Revenue (3-Year)', '% Change in EBITDA (1-Year)', '% Change in EBITDA (3-Year)']:
    df[column].fillna(df[column].mean(), inplace = True)

df

,Ticker,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,D/E Ratio,D/E Percentile,% Change in Revenue (1-Year),1 Year Revenue Change Percentile,% Change in Revenue (3-Year),3 Year Revenue Change Percentile,% Change in EBITDA (1-Year),1 Year EBITDA Change Percentile,% Change in EBITDA (3-Year),3 Year EBITDA Change Percentile,Value Score,Growth Score,Total Score
0,AAPL,28.700000,N/A,38.1,N/A,7.2,N/A,1.57,N/A,5.512080,N/A,19.753178,N/A,-1.026142,N/A,5.813058,N/A,N/A,N/A,N/A
1,MSFT,39.000000,N/A,16.0,N/A,13.7,N/A,0.44,N/A,13.645574,N/A,58.993885,N/A,17.856897,N/A,100.366043,N/A,N/A,N/A,N/A
2,AMZN,68.900000,N/A,17.7,N/A,4.4,N/A,0.82,N/A,37.623431,N/A,117.053287,N/A,36.694768,N/A,216.612943,N/A,N/A,N/A,N/A
3,FB,5.800000,N/A,1.1,N/A,5.7,N/A,7.08,N/A,-0.458365,N/A,16.819980,N/A,24.411729,N/A,45.485726,N/A,N/A,N/A,N/A
4,GOOGL,35.200000,N/A,7.7,N/A,9.8,N/A,0.11,N/A,12.770532,N/A,64.653827,N/A,20.207354,N/A,80.945144,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,FOX,26.000000,N/A,1.9,N/A,2.8,N/A,0.25,N/A,10.661698,N/A,43.167921,N/A,32.349949,N/A,130.469442,N/A,N/A,N/A,N/A
501,DISCA,19.600000,N/A,1.3,N/A,1.8,N/A,1.36,N/A,-4.244436,N/A,55.259712,N/A,-6.693627,N/A,159.542281,N/A,N/A,N/A,N/A
502,UAA,78.200000,N/A,5.3,N/A,1.9,N/A,1.02,N/A,-15.037023,N/A,-10.086397,N/A,24.411729,N/A,45.485726,N/A,N/A,N/A,N/A
503,UA,67.500000,N/A,4.5,N/A,1.7,N/A,1.02,N/A,-15.037023,N/A,-10.086397,N/A,24.411729,N/A,45.485726,N/A,N/A,N/A,N/A


### Calculate Scoring Criteria

In [7]:
# Calculate percentile values

from scipy.stats import percentileofscore as score
value_metrics = {
    'P/E Ratio' : 'P/E Percentile',
    'P/B Ratio' : 'P/B Percentile',
    'P/S Ratio' : 'P/S Percentile',
    'D/E Ratio' : 'D/E Percentile'
}

growth_metrics = {
    '% Change in Revenue (1-Year)' : '1 Year Revenue Change Percentile',
    '% Change in Revenue (3-Year)' : '3 Year Revenue Change Percentile',
    '% Change in EBITDA (1-Year)' : '1 Year EBITDA Change Percentile',
    '% Change in EBITDA (3-Year)' : '3 Year EBITDA Change Percentile'
}

for metric in value_metrics.keys():
    for row in df.index:
        df.loc[row, value_metrics[metric]] = 100 - score( df[metric], df.loc[row, metric])
for metric in growth_metrics.keys():
    for row in df.index:
        df.loc[row, growth_metrics[metric]] = score( df[metric], df.loc[row, metric])
        
df

,Ticker,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,D/E Ratio,D/E Percentile,% Change in Revenue (1-Year),1 Year Revenue Change Percentile,% Change in Revenue (3-Year),3 Year Revenue Change Percentile,% Change in EBITDA (1-Year),1 Year EBITDA Change Percentile,% Change in EBITDA (3-Year),3 Year EBITDA Change Percentile,Value Score,Growth Score,Total Score
0,AAPL,28.700000,55.049505,38.1,3.366337,7.2,23.465347,1.57,27.623762,5.512080,71.287129,19.753178,64.554455,-1.026142,35.445545,5.813058,31.881188,N/A,N/A,N/A
1,MSFT,39.000000,40.792079,16.0,11.683168,13.7,9.207921,0.44,75.544554,13.645574,85.148515,58.993885,90.49505,17.856897,60.594059,100.366043,93.465347,N/A,N/A,N/A
2,AMZN,68.900000,11.881188,17.7,10.29703,4.4,40.39604,0.82,53.267327,37.623431,97.425743,117.053287,97.623762,36.694768,89.70297,216.612943,98.415842,N/A,N/A,N/A
3,FB,5.800000,99.306931,1.1,95.643564,5.7,32.277228,7.08,4.950495,-0.458365,48.514851,16.819980,58.415842,24.411729,74.851485,45.485726,70.693069,N/A,N/A,N/A
4,GOOGL,35.200000,46.435644,7.7,31.584158,9.8,15.544554,0.11,96.138614,12.770532,84.455446,64.653827,91.782178,20.207354,62.871287,80.945144,90.792079,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,FOX,26.000000,60.09901,1.9,79.50495,2.8,58.514851,0.25,88.217822,10.661698,81.782178,43.167921,84.950495,32.349949,87.722772,130.469442,95.247525,N/A,N/A,N/A
501,DISCA,19.600000,75.346535,1.3,91.683168,1.8,73.762376,1.36,32.178218,-4.244436,34.950495,55.259712,89.60396,-6.693627,28.415842,159.542281,96.732673,N/A,N/A,N/A
502,UAA,78.200000,8.910891,5.3,43.465347,1.9,71.782178,1.02,42.970297,-15.037023,12.970297,-10.086397,16.930693,24.411729,74.851485,45.485726,70.693069,N/A,N/A,N/A
503,UA,67.500000,13.069307,4.5,49.80198,1.7,76.336634,1.02,42.970297,-15.037023,12.970297,-10.086397,16.930693,24.411729,74.851485,45.485726,70.693069,N/A,N/A,N/A


In [8]:
# Specify weight of value & growth criteria (value_weight & growth_weight should add to 1.o to keep 100.0 scale for total score)
value_weight = .20
growth_weight = .80

# Calculate Value, Growth, & Total Scores
for row in df.index:
    value_percentiles = []
    growth_percentiles = []
    for metric in value_metrics.keys():
        value_percentiles.append(df.loc[row, value_metrics[metric]])
    for metric in growth_metrics.keys():
        growth_percentiles.append(df.loc[row, growth_metrics[metric]])
    df.loc[row, 'Value Score'] = mean(value_percentiles)
    df.loc[row, 'Growth Score'] = mean(growth_percentiles)
    df.loc[row, 'Total Score'] = (value_weight * df.loc[row, 'Value Score']) + (growth_weight * df.loc[row, 'Growth Score'])

df

,Ticker,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,D/E Ratio,D/E Percentile,% Change in Revenue (1-Year),1 Year Revenue Change Percentile,% Change in Revenue (3-Year),3 Year Revenue Change Percentile,% Change in EBITDA (1-Year),1 Year EBITDA Change Percentile,% Change in EBITDA (3-Year),3 Year EBITDA Change Percentile,Value Score,Growth Score,Total Score
0,AAPL,28.700000,55.049505,38.1,3.366337,7.2,23.465347,1.57,27.623762,5.512080,71.287129,19.753178,64.554455,-1.026142,35.445545,5.813058,31.881188,27.376238,50.792079,46.108911
1,MSFT,39.000000,40.792079,16.0,11.683168,13.7,9.207921,0.44,75.544554,13.645574,85.148515,58.993885,90.49505,17.856897,60.594059,100.366043,93.465347,34.306931,82.425743,72.80198
2,AMZN,68.900000,11.881188,17.7,10.29703,4.4,40.39604,0.82,53.267327,37.623431,97.425743,117.053287,97.623762,36.694768,89.70297,216.612943,98.415842,28.960396,95.792079,82.425743
3,FB,5.800000,99.306931,1.1,95.643564,5.7,32.277228,7.08,4.950495,-0.458365,48.514851,16.819980,58.415842,24.411729,74.851485,45.485726,70.693069,58.044554,63.118812,62.10396
4,GOOGL,35.200000,46.435644,7.7,31.584158,9.8,15.544554,0.11,96.138614,12.770532,84.455446,64.653827,91.782178,20.207354,62.871287,80.945144,90.792079,47.425743,82.475248,75.465347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,FOX,26.000000,60.09901,1.9,79.50495,2.8,58.514851,0.25,88.217822,10.661698,81.782178,43.167921,84.950495,32.349949,87.722772,130.469442,95.247525,71.584158,87.425743,84.257426
501,DISCA,19.600000,75.346535,1.3,91.683168,1.8,73.762376,1.36,32.178218,-4.244436,34.950495,55.259712,89.60396,-6.693627,28.415842,159.542281,96.732673,68.242574,62.425743,63.589109
502,UAA,78.200000,8.910891,5.3,43.465347,1.9,71.782178,1.02,42.970297,-15.037023,12.970297,-10.086397,16.930693,24.411729,74.851485,45.485726,70.693069,41.782178,43.861386,43.445545
503,UA,67.500000,13.069307,4.5,49.80198,1.7,76.336634,1.02,42.970297,-15.037023,12.970297,-10.086397,16.930693,24.411729,74.851485,45.485726,70.693069,45.544554,43.861386,44.19802


### Results

In [9]:
# Sort by total score, extract top 50 stocks

df.sort_values('Total Score', ascending=False, inplace = True)
final_df = df[:50]
final_df

,Ticker,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,D/E Ratio,D/E Percentile,% Change in Revenue (1-Year),1 Year Revenue Change Percentile,% Change in Revenue (3-Year),3 Year Revenue Change Percentile,% Change in EBITDA (1-Year),1 Year EBITDA Change Percentile,% Change in EBITDA (3-Year),3 Year EBITDA Change Percentile,Value Score,Growth Score,Total Score
173,CNC,58.700000,15.643564,1.6,86.237624,0.4,98.316832,0.690000,59.70297,48.869894,98.217822,129.661858,97.821782,67.782869,95.049505,172.000000,97.227723,64.975248,97.079208,90.658416
164,LHX,35.400000,46.039604,2.3,73.465347,2.7,60.792079,0.380000,79.70297,167.519482,100.0,208.372881,99.60396,70.389610,95.445545,89.048991,92.277228,65.0,96.831683,90.465347
147,VRTX,19.300000,75.940594,5.8,40.0,8.3,18.712871,0.100000,96.534653,49.075186,98.415842,149.337083,98.613861,110.512821,97.425743,3088.349515,99.80198,57.79703,98.564356,90.410891
377,PKI,16.200000,82.970297,4.5,49.80198,3.9,44.950495,0.600000,65.841584,31.171983,95.643564,67.611874,92.277228,133.592233,98.019802,169.730942,97.029703,60.891089,95.742574,88.772277
243,DHI,9.300000,95.247525,2.4,72.079208,1.3,84.554455,0.320000,84.257426,15.449326,86.930693,44.141651,85.742574,40.672348,90.891089,82.158185,91.287129,84.034653,88.712871,87.777228
122,FISV,68.300000,12.475248,2.4,72.079208,5.0,36.930693,0.650000,61.683168,45.793659,98.019802,160.744382,98.811881,58.715310,94.257426,158.769700,96.435644,45.792079,96.881188,86.663366
361,IR,150.700000,2.574257,2.3,73.465347,3.6,48.217822,0.430000,76.237624,100.244698,99.60396,106.736842,97.227723,29.475983,86.732673,193.564356,98.019802,50.123762,95.39604,86.341584
73,AMD,38.400000,42.079208,17.1,10.891089,9.6,15.841584,0.090000,96.930693,45.045313,97.821782,83.205104,94.851485,131.491713,97.821782,394.395280,99.009901,41.435644,97.376238,86.188119
136,GPN,90.500000,6.534653,2.1,75.742574,7.7,20.792079,0.360000,81.584158,51.140065,98.613861,86.767296,95.445545,50.029394,92.673267,150.441609,95.841584,46.163366,95.643564,85.747525
268,LH,10.500000,92.673267,2.8,67.920792,1.8,73.762376,0.560000,67.920792,20.977932,90.693069,33.885643,78.613861,58.658346,94.059406,60.325801,86.336634,75.569307,87.425743,85.054455
